In [1]:
from chunkformer_vpb.training.finetune_config import FinetuneConfig
from chunkformer_vpb.training.optimizer import build_model_and_optimizer
import torch

# Load config
cfg_path = "../../config/vpb_finetune_config.yaml"  # đổi path đúng
cfg = FinetuneConfig.from_yaml(cfg_path)

# Init model như khi train (không cần optimizer, scheduler nếu chỉ test)
device = "cpu"
model, tokenizer, _, _ = build_model_and_optimizer(cfg, device, total_steps=100)  # total_steps chỉ là placeholder
model.to(device)


/home/kylh/.local/share/mamba/envs/stt310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-07-22 13:57:13] INFO: Checkpoint: loading from checkpoint ../../../chunkformer-large-vie/pytorch_model.bin for GPU



🧾 Loaded checkpoint from: ../../../chunkformer-large-vie/pytorch_model.bin
📦 Checkpoint keys: ['encoder.global_cmvn.mean', 'encoder.global_cmvn.istd', 'encoder.embed.out.weight', 'encoder.embed.out.bias', 'encoder.embed.conv.0.weight'] ... (total 813)
🔍 AED decoder head included in checkpoint? ✅ YES
📊 Model total params: 113,852,240, trainable: 113,852,240
!!!Vocab size: 6992


ASRModel(
  (encoder): ChunkFormerEncoder(
    (global_cmvn): GlobalCMVN()
    (embed): DepthwiseConvSubsampling(
      (pos_enc): StreamingRelPositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (out): Linear(in_features=4608, out_features=512, bias=True)
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), groups=512)
        (3): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (4): ReLU()
        (5): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), groups=512)
        (6): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (7): ReLU()
      )
    )
    (after_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (encoders): ModuleList(
      (0-16): 17 x ChunkFormerEncoderLayer(
        (self_attn): ChunkAttentionWithRelativeRightContext(
          (linear_q): Linear(in_features=512, out_features=51

In [2]:
ckpt_path = "checkpoints_vpb/epoch1.pt"
state_dict = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state_dict)
print(f"✅ Loaded checkpoint from {ckpt_path}")


✅ Loaded checkpoint from checkpoints_vpb/epoch1.pt


In [3]:
from chunkformer_vpb.training.train import evaluate
from chunkformer_vpb.training.data_loader import get_dataloaders

train_loader, dev_loader = get_dataloaders(cfg)
evaluate(model, tokenizer, dev_loader, cfg, device)


../../../vpb_dataset
../../../vpb_dataset
🎯 Dev WER (CTC): 23.68%
🌐 Global WER           : 14.90%


In [4]:
evaluate(model, tokenizer, train_loader, cfg, device)

🎯 Dev WER (CTC): 20.44%
🌐 Global WER           : 12.48%
